In [1]:
from datetime import datetime
import numpy as np
import geopandas
import pyproj
import shapely.geometry

In [2]:
file = 'modis_pasture_A2017_square1_d_sq.txt'
modisData, transpose = [], []

modisFile = open(file, 'r')

transition = []

for line in modisFile.readlines():

    line = line.replace('"',  '')
    line = line.replace('\n', '')
    line = line.split(' ')

    if len(transition) == 0:
        line.insert(0, 'id')
    else:
        line[0] = int(line[0])
        line[1] = float(line[1])
        line[2] = float(line[2])
        line[3] = float(line[3])
        line[4] = float(line[4])
        line[6] = int(line[6])
        line[7] = int(line[7])
        line[8] = int(line[8])
        line[9] = float(line[9])
        line[10] = float(line[10])
        line[11] = int(line[11])
        line[12] = datetime.strptime(line[12], '%Y-%m-%d')

    transition.append(line)

modisFile.close()

header = transition[0]
transition = sorted (transition[1::], key=lambda day: day[8])
transition = np.insert(transition, 0, header, axis=0)

modisData = transition

transpose.append(transition[1::].T)
xMODIS = transpose[-1][3]
yMODIS = transpose[-1][4]
coordMODIS = np.array([xMODIS,yMODIS]).T

print (modisData)

[['id' 'x_ITM' 'y_ITM' ... 'ndvi' 'QC' 'date']
 [11 714443.370165114 761937.917724095 ... 0.669 1
  datetime.datetime(2017, 1, 1, 0, 0)]
 [12 714675.717152128 761943.567815079 ... 0.6735 1
  datetime.datetime(2017, 1, 1, 0, 0)]
 ...
 [181035 713912.47234178 752183.961850394 ... 0.6213 1
  datetime.datetime(2017, 12, 27, 0, 0)]
 [182741 717862.346898622 752280.817223724 ... 0.7633 1
  datetime.datetime(2017, 12, 27, 0, 0)]
 [182842 718094.69131406 752286.617449876 ... 0.7664 1
  datetime.datetime(2017, 12, 27, 0, 0)]]


In [3]:
gpd = geopandas.read_file('HNVf-Simplified.shp')
print (gpd)

                EDNAME  HNVf_likel  \
0     Balbriggan Rural    3.187518   
1         Kilsallaghan    3.210960   
2                 Lusk    3.316044   
3          Ballyboghil    3.229751   
4           Balscadden    3.137701   
..                 ...         ...   
200         Castlewray    3.685656   
201        Edenacarnan    4.043524   
202         Gortnavern    4.063523   
203  Letterkenny Rural    4.017301   
204    Manorcunningham    3.210416   

                                              geometry  
0    POLYGON ((318277.159 266344.065, 318428.675 26...  
1    POLYGON ((313257.642 251715.874, 313448.281 25...  
2    POLYGON ((321393.910 251839.776, 321354.190 25...  
3    POLYGON ((313132.008 256181.718, 313149.044 25...  
4    POLYGON ((316604.472 266295.848, 316570.335 26...  
..                                                 ...  
200  POLYGON ((223908.382 419042.447, 223957.656 41...  
201  POLYGON ((214215.221 419636.116, 214305.042 41...  
202  POLYGON ((217512.660 4200

In [4]:
# https://spatialreference.org/ref/sr-org/modis-sinusoidal-3/ for MODIS
projMODIS = pyproj.Proj("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

# Irish National Grid, epsg:29901
# TM65 / Irish Grid, epsg:29902
# TM75 / Irish Grid, epsg:29903
# IRENET95 / Irish Transverse Mercator, epsg:2157
# IRENET95 / UTM zone 29N, epsg:2158
projOut = pyproj.Proj(init='epsg:29902')

In [5]:
hnv = []

for xy in zip(xMODIS, yMODIS):
    xProj, yProj = pyproj.transform(projMODIS, projOut, xy[0], xy[1])
    
    point = shapely.geometry.Point(xProj, yProj)
    
    find = False
    
    for polygon in gpd.values:
        
        if point.within(polygon[2]):
            value = polygon[1]
            find = True
            break
        
    if find:
        hnv.append(value)
    else:
        hnv.append(-1)

print (set(hnv))

{2.85216510619, 3.13770094875, 3.18751786, 3.27415708444, 3.06297827375, 3.31604436267, 3.22975107364, 3.21096025455}
